# Test hypotézy
## Pokud viník nehody byl pod silným vlivem alkoholu, došlo častěji k těžkým zdravotním následkům.

K overeniu hypotézy použijeme $\chi^2$ test.

Import potrebných knižníc, `numpy` pre výpočty, `seaborn` a `matplotlib.pyplot` pre vizualizáciu, `scipy.stats` pre štatistickú analýzu

In [1]:
import numpy as np, pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt

Načítame data zo súboru. Pozrieme sa na data.

In [2]:
df = pd.read_pickle("accidents.pkl.gz")
df

,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,,2016-01-01,5,0055,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,,2016-01-01,5,0130,1,3,0,2,...,,,,,,,,,1,PHA
2,002100160003,5,,2016-01-01,5,0100,1,2,0,2,...,,,,,,,,,1,PHA
3,002100160004,6,,2016-01-01,5,0120,9,0,0,2,...,,,,,,,,,1,PHA
4,002100160005,6,,2016-01-01,5,2560,2,0,0,2,...,,,,,,,,,1,PHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487156,190906200498,6,,2020-09-29,2,2560,2,0,0,2,...,,3438492,,Souhlasnýsesměremúseku,Pomalý,560294,,GN_V0.1UIR-ADR_410,1,KVK
487157,190906200499,0,6,2020-09-29,2,1940,1,2,0,2,...,D6,1671277,,Souhlasnýsesměremúseku,Pomalý,560286,,GN_V0.1UIR-ADR_410,2,KVK
487158,190906200500,3,2114,2020-09-30,3,2045,1,4,0,2,...,2114,1669033,,Opačnýkesměruúseku,Pomalý,554642,,GN_V0.1UIR-ADR_410,2,KVK
487159,190906200501,0,6,2020-10-01,4,0420,3,0,4,2,...,D6,3513947,,Souhlasnýsesměremúseku,Pomalý,560294,,GN_V0.1UIR-ADR_410,2,KVK


Máme v nich veľa nepotrebných informácii pre našu štatistickú analýzu, preto si vyberieme iba dôležité, teda stĺpec `p11`, obsahujúci informáciu o prítomnosti alkoholu u vinníka nehody. U neho nás zaujíma jú hodnoty väčšie a rovné ako 7, ktoré predstavujú obsah alkoholu v krvi nad 0.8‰. Nehody, kde má tento sĺpec stĺpec hodnotu 4 alebo 5 úplne vylúčime, pretože tieto predstavujú prítomnosť drôg a tuna skúmame len silný vplyv alkoholu. Stĺpce `p13a` a `p13b`, reprezentujúce usmrtenie a ťažké zranenie spôsobené nehodôv, budeme oba považovať za ťažké zdravotné následky.

Vytvoríme nový `DataFrame` s potrebnými dátami a odstránime tie riadky, kde chýbajú niektoré z pre nás dôležitých hodnôt.

In [3]:
df = df[(df.p11 != 4) & (df.p11 != 5)]
df_clear = pd.DataFrame({
          'alcohol': df['p11'] >= 7,
          'injury': (df['p13a'] + df['p13b']) > 0,
          'alcoinjury': (df['p11'] >= 7) & (df['p13a'] | df['p13b'])
         }).dropna()
df_clear

,alcohol,injury,alcoinjury
0,False,False,False
1,False,False,False
2,False,False,False
3,True,False,False
4,False,False,False
...,...,...,...
487156,False,False,False
487157,False,False,False
487158,False,False,False
487159,False,False,False


## $\chi^2$ test hypotézy
Overíme štatistickú koreláciu $\chi^2$ testom. Dáta už máme v binárnom formáte z predošlého spracovania.

Ďalej vytvoríme kontingenčú tabuľku pre všetky kombinácie vstupov.

In [4]:
ct = pd.crosstab(df_clear['alcohol'], df_clear['injury'])
ct

injury,False,True
alcohol,,
False,457528,10777
True,16492,886


Vyrátame samotný $\chi^2$ test pomocou funkcie `scipy.stats.chi2_contingency()`, ktorá vráti hodnoty (xi2, p-value, DF, E). Z nich nás zaujíma p-value.

In [5]:
st.chi2_contingency(ct)

(558.1749514234125,
 2.0971505700338304e-123,
 1,
 array([[4.57059308e+05, 1.12456916e+04],
        [1.69606916e+04, 4.17308438e+02]]))

## Záver
P-hodnota je menšia $2.097\cdot10^{-123}$ je výrazne menšia ako $0.05$. Preto zamietame nulovú hypotézu a môžme s 95% istotou prehlásiť, že je štatisticky významná korelácia medzi prítomnosťou vysokého obsahu alkoholu v krvi vodiča a ťažkými zraneniami spôsobenými v autonehode, teda potvrdzujeme testovanú hypotézu.